# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [68]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [69]:
cities_df =  pd.read_csv('../output_data/cities.csv')
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hithadhoo,-0.6000,73.0833,82.31,64,10,4.83,MV,1619476755
1,mersing,2.4312,103.8405,73.40,100,59,3.44,MY,1619476755
2,quatre cocos,-20.2078,57.7625,78.80,83,75,12.66,MU,1619476756
3,dikson,73.5069,80.5464,31.82,98,100,17.09,RU,1619476756
4,nikolskoye,59.7035,30.7861,41.00,75,75,8.95,RU,1619476574
...,...,...,...,...,...,...,...,...,...
561,kudat,6.8837,116.8477,75.20,100,20,3.11,MY,1619477025
562,coquimbo,-29.9533,-71.3436,60.80,72,20,9.22,CL,1619477025
563,saiha,22.4833,92.9667,67.01,59,4,3.29,IN,1619477026
564,karakol,42.4907,78.3936,44.31,32,51,6.87,KG,1619477026


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [70]:
gmaps.configure(api_key=g_key)

In [71]:
locations = cities_df[['Lat','Lng']]
humidity = cities_df['Humidity']
fig = gmaps.figure()
max_humidity = humidity.max().astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=max_humidity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [72]:
filtered_df = cities_df[(cities_df['Max Temp']>70)&(cities_df['Max Temp']<80)]
filtered_df = filtered_df[filtered_df['Wind Speed']<10]
filtered_df = filtered_df[filtered_df['Cloudiness']==0]
filtered_df = filtered_df.dropna()
filtered_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
96,francisco sa,-16.4758,-43.4883,73.40,78,0,3.44,BR,1619476799
102,karratha,-20.7377,116.8463,74.84,33,0,9.01,AU,1619476689
225,yanam,16.7333,82.2167,76.77,84,0,2.55,IN,1619476863
275,dawlatabad,36.4141,64.9053,70.48,24,0,0.09,AF,1619476887
321,brufut,13.3814,-16.7518,71.60,78,0,6.91,GM,1619476909
371,dzaoudzi,-12.7887,45.2699,78.80,89,0,2.30,YT,1619476933


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [73]:
data ={
    'Hotel Name': [],
    'Lat' : [],
    'Lng' : [],
    'City': [],
    'Country': []
}
for index, row in filtered_df.iterrows():
    print(f'Requesting data for {row["City"]}')
    params = {
        "location": f"{row['Lat']}, {row['Lng']}",
        "keyword": "hotel","radius": 5000,
        "type": "lodging","key": g_key}
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    try:
        data['Hotel Name'].append(response["results"][0]["name"])
        data['Lat'].append(response["results"][0]["geometry"]["location"]["lat"])
        data['Lng'].append(response["results"][0]["geometry"]["location"]["lng"])
        data['City'].append(row['City'])
        data['Country'].append(row['Country'])
        print('Done')        
    except:
        print(f'Incomplete data, Skiping ...')
hotel_df = pd.DataFrame.from_dict(data)
hotel_df

Requesting data for francisco sa
Done
Requesting data for karratha
Done
Requesting data for yanam
Done
Requesting data for dawlatabad
Incomplete data, Skiping ...
Requesting data for brufut
Done
Requesting data for dzaoudzi
Done


,Hotel Name,Lat,Lng,City,Country
0,Village Hotel e Eventos Ltda Me,-16.475144,-43.498724,francisco sa,BR
1,Karratha International Hotel,-20.737639,116.841998,karratha,AU
2,The Regency Hotel,16.732043,82.202045,yanam,IN
3,Coco Ocean Resort & Spa,13.425278,-16.732778,brufut,GM
4,Hotel Maharajah Mayotte,-12.782349,45.229721,dzaoudzi,YT


In [74]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [63]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))